In [ ]:
# 这个notebook 介绍了 如何对split learning 发起 inverse-model attack攻击

In [1]:
# 导包
import sys
sys.path.append('/home/dengruijun/data/FinTech/PP-Split/')
from ppsplit.attacks.model_inversion.inverse_model import InverseModelAttack
import torch
import os


# cifar10

In [2]:
from target_model.data_preprocessing.preprocess_cifar10 import get_cifar10_normalize,get_one_data,deprocess
from target_model.models.VGG import VGG,VGG5Decoder,model_cfg
from target_model.models.splitnn_utils import split_weights_client

In [6]:
# 准备基本模型client net
client_net_route = '/home/dengruijun/data/project/Inverse_efficacy/trained_models/VGG5/BN+Tanh/VGG5-20ep.pth'
save_inverse_dir = '../results/VGG5/inverted/'


# VGG5 完整的edge net
split_layer_list = list(range(len(model_cfg['VGG5'])))

# 切割成client-server
split_layer = 1
client_net = VGG('Client','VGG5',split_layer,model_cfg)
pweights = torch.load(client_net_route)
if split_layer < len(model_cfg['VGG5']):
    pweights = split_weights_client(pweights,client_net.state_dict())
client_net.load_state_dict(pweights)



features.0.weight
features.0.bias
features.1.weight
features.1.bias
features.1.running_mean
features.1.running_var
features.1.num_batches_tracked
=> loading decoder mode '../results/VGG5-raw_layer1_decoder.pth'


In [ ]:
# 准备inverse_model attack使用到的东西
# decoder net
decoder_net_route = '../results/VGG5-raw_layer1_decoder.pth'
if os.path.isfile(decoder_net_route):
    print("=> loading decoder mode '{}'".format(decoder_net_route))
    decoder_net = torch.load(decoder_net_route)
else:
    decoder_net = VGG5Decoder(split_layer=1)

# 定义inverse atatck对象
im_attack = InverseModelAttack(decoder_route=decoder_net_route,data_type=1,inverse_dir=save_inverse_dir)

In [4]:
# 训练decoder
trainloader,testloader = get_cifar10_normalize(batch_size=32)

decoder_net= im_attack.train_decoder(client_net=client_net,decoder_net=decoder_net,
                        train_loader=trainloader,test_loader=testloader,
                        epochs=20)


  0%|          | 0/1563 [00:00<?, ?it/s]

----train decoder----
client_net: 
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (denses): Sequential()
)
decoder_net: 
VGG5Decoder(
  (features): Sequential(
    (0): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
  )
  (denses): Sequential()
)
Epoch 0


  0%|          | 0/1563 [00:00<?, ?it/s]

--- epoch: 0, train_loss: [0.26835113763809204]
Epoch 1


  0%|          | 0/1563 [00:00<?, ?it/s]

...

--- epoch: 17, train_loss: [0.050050705671310425]
Epoch 18


  0%|          | 0/1563 [00:00<?, ?it/s]

--- epoch: 18, train_loss: [0.04929197579622269]
Epoch 19


100%|██████████| 1563/1563 [00:13<00:00, 118.41it/s]

--- epoch: 19, train_loss: [0.04860677570104599]
model saved


In [8]:
# 实现攻击
trainloader,testloader = get_cifar10_normalize(batch_size=1)

im_attack.inverse(client_net=client_net,decoder_net=decoder_net,
                  train_loader=trainloader,test_loader=testloader,
                  deprocess=deprocess,
                  save_fake=True)

  0%|          | 0/10000 [00:00<?, ?it/s]

----train decoder----
client_net: 
VGG(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (denses): Sequential()
)
decoder_net: 
VGG5Decoder(
  (features): Sequential(
    (0): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
  )
  (denses): Sequential()
)


100%|██████████| 10000/10000 [00:50<00:00, 199.57it/s]


SSIM: 0.6909938108563423,              MSE:0.11700416839849204


# Bank数据集 